In [ ]:
# Cell 1: Install dependencies and setup for YOLOv5
import os

# Clone YOLOv5 if not already cloned
if not os.path.exists('/content/yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git /content/yolov5
    print("YOLOv5 cloned successfully!")
else:
    print("YOLOv5 already exists!")

%cd /content/yolov5

# Install requirements
!pip install -qr requirements.txt

# Additional packages
!pip install -q kaggle

# Additional imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files, drive
import shutil
from pathlib import Path
import yaml
import pandas as pd
import torch

print("All dependencies installed successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Cell 2: Mount Drive and setup paths
drive.mount('/content/drive')

local_dataset_path = '/content/Bangla_License_Plate_Dataset'
yolo_dataset_path = '/content/yolo_dataset'
drive_backup_path = '/content/drive/MyDrive/Bangla_License_Plate_Dataset'

print("Drive mounted successfully!")

# Cell 3: Download and prepare dataset (same as original)
dataset_ready = False

if os.path.exists(local_dataset_path):
    print("Dataset already exists locally")
    dataset_ready = True
elif os.path.exists(drive_backup_path):
    print("Copying dataset from Drive to local storage...")
    shutil.copytree(drive_backup_path, local_dataset_path)
    print("Dataset copied to local storage")
    dataset_ready = True
else:
    print("Dataset not found. Please upload your kaggle.json file:")
    uploaded = files.upload()

    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    os.chmod('/root/.kaggle/kaggle.json', 0o600)

    print("Downloading dataset from Kaggle...")
    os.system('kaggle datasets download -d nishat99/bangla-license-plate-detection -p /content')
    os.system('unzip -q /content/bangla-license-plate-detection.zip -d /content')

    if os.path.exists('/content/Bangla License Plate Dataset'):
        shutil.move('/content/Bangla License Plate Dataset', local_dataset_path)
        dataset_ready = True

        try:
            shutil.copytree(local_dataset_path, drive_backup_path)
            print("Dataset backed up to Drive")
        except:
            print("Drive backup failed, continuing with local dataset")

if not dataset_ready:
    print("ERROR: Dataset preparation failed!")
else:
    print(f"Dataset ready at: {local_dataset_path}")

# Cell 4: Convert annotations to YOLO format (same as original)
def mask_to_bbox(mask):
    """Convert binary mask to bounding box coordinates"""
    coords = np.where(mask > 127)
    if len(coords[0]) == 0:
        return []
    y_min, y_max = coords[0].min(), coords[0].max()
    x_min, x_max = coords[1].min(), coords[1].max()
    return [(x_min, y_min, x_max, y_max)]

def bbox_to_yolo_format(bbox, img_width, img_height):
    """Convert bounding box to YOLO format (normalized)"""
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min

    x_center_norm = x_center / img_width
    y_center_norm = y_center / img_height
    width_norm = width / img_width
    height_norm = height / img_height

    return f"0 {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}"

def process_dataset_split(split_name, local_dataset_path, yolo_dataset_path):
    """Process one split with progress tracking"""
    img_folder = os.path.join(local_dataset_path, split_name, 'img')
    mask_folder = os.path.join(local_dataset_path, split_name, 'masks')

    print(f"\nProcessing {split_name}:")

    if not os.path.exists(img_folder) or not os.path.exists(mask_folder):
        print(f"ERROR: Missing folders for {split_name}")
        return 0

    yolo_img_dir = os.path.join(yolo_dataset_path, split_name, 'images')
    yolo_label_dir = os.path.join(yolo_dataset_path, split_name, 'labels')
    os.makedirs(yolo_img_dir, exist_ok=True)
    os.makedirs(yolo_label_dir, exist_ok=True)

    img_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(img_files)} images in {split_name}")

    processed_count = 0

    for i, img_file in enumerate(img_files):
        if i % 500 == 0 and i > 0:
            print(f"  Processed {i}/{len(img_files)} images ({i/len(img_files)*100:.1f}%)")

        try:
            mask_name = os.path.splitext(img_file)[0] + '.png'
            mask_path = os.path.join(mask_folder, mask_name)

            if not os.path.exists(mask_path):
                continue

            mask = cv2.imread(mask_path, 0)
            if mask is None:
                continue

            img_path = os.path.join(img_folder, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img_height, img_width = img.shape[:2]
            bboxes = mask_to_bbox(mask)

            # Copy image
            dst_img_path = os.path.join(yolo_img_dir, img_file)
            shutil.copy2(img_path, dst_img_path)

            # Create label file
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(yolo_label_dir, label_file)

            with open(label_path, 'w') as f:
                for bbox in bboxes:
                    yolo_line = bbox_to_yolo_format(bbox, img_width, img_height)
                    f.write(yolo_line + '\n')

            processed_count += 1

        except Exception as e:
            print(f"Error processing {img_file}: {e}")
            continue

    print(f"{split_name.upper()} processed: {processed_count} images")
    return processed_count

# Check if annotations already converted
annotations_complete_flag = os.path.join(yolo_dataset_path, 'annotations_complete.txt')

if os.path.exists(annotations_complete_flag):
    print("Annotations already exist! Skipping annotation generation...")
else:
    print("Creating YOLO annotations from masks...")

    train_count = process_dataset_split('train', local_dataset_path, yolo_dataset_path)
    val_count = process_dataset_split('validation', local_dataset_path, yolo_dataset_path)
    test_count = process_dataset_split('test', local_dataset_path, yolo_dataset_path)

    total_count = train_count + val_count + test_count
    print(f"\nANNOTATION CONVERSION COMPLETED!")
    print(f"Train: {train_count}, Validation: {val_count}, Test: {test_count}")
    print(f"Total: {total_count} images")

    with open(annotations_complete_flag, 'w') as f:
        f.write(f"Total: {total_count} images")

# Cell 5: Create dataset configuration for YOLOv5
dataset_config = {
    'path': yolo_dataset_path,
    'train': 'train/images',
    'val': 'validation/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['license_plate']
}

config_path = os.path.join(yolo_dataset_path, 'dataset.yaml')
with open(config_path, 'w') as f:
    yaml.dump(dataset_config, f)

print("Dataset configuration created!")
print(f"Config saved at: {config_path}")

In [ ]:
# Cell 6: YOLOv5m Single-Stage Training with Drive Checkpoint Backup
import torch
import os
import subprocess
import shutil

# Make sure we're in the YOLOv5 directory
%cd /content/yolov5

# Setup paths - MODIFIED TO USE DRIVE
model_save_dir = '/content/yolov5_models'  # Local training directory
drive_checkpoint_dir = '/content/drive/MyDrive/yolov5_checkpoints'  # Drive backup

# Create directories
os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(drive_checkpoint_dir, exist_ok=True)

print(f"GPU Available: {torch.cuda.is_available()}")

# Check GPU memory and set conservative batch size
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU Memory: {gpu_memory:.1f} GB")

    if gpu_memory < 12:
        recommended_batch = 4
    elif gpu_memory < 16:
        recommended_batch = 8
    else:
        recommended_batch = 16
else:
    recommended_batch = 4

print(f"Using batch size: {recommended_batch}")

# VERIFY DATASET BEFORE TRAINING
print("\n" + "="*60)
print("DATASET VERIFICATION")
print("="*60)
print(f"Config path: {config_path}")
print(f"Config exists: {os.path.exists(config_path)}")

train_imgs = os.path.join(yolo_dataset_path, 'train/images')
val_imgs = os.path.join(yolo_dataset_path, 'validation/images')

print(f"\nTrain images path: {train_imgs}")
print(f"Train images exist: {os.path.exists(train_imgs)}")
if os.path.exists(train_imgs):
    train_count = len([f for f in os.listdir(train_imgs) if f.endswith(('.jpg', '.png'))])
    print(f"Number of train images: {train_count}")

print(f"\nValidation images path: {val_imgs}")
print(f"Validation images exist: {os.path.exists(val_imgs)}")
if os.path.exists(val_imgs):
    val_count = len([f for f in os.listdir(val_imgs) if f.endswith(('.jpg', '.png'))])
    print(f"Number of validation images: {val_count}")

# Read and display dataset config
print("\nDataset Configuration:")
with open(config_path, 'r') as f:
    print(f.read())

# CHECK FOR EXISTING CHECKPOINT - TRY DRIVE FIRST, THEN LOCAL
drive_checkpoint_path = os.path.join(drive_checkpoint_dir, 'last.pt')
local_checkpoint_path = os.path.join(model_save_dir, 'single_stage', 'weights', 'last.pt')

resume_training = False
checkpoint_to_use = None

# Priority: Check Drive first (in case of runtime restart)
if os.path.exists(drive_checkpoint_path):
    print(f"\n✓ Found checkpoint in Drive: {drive_checkpoint_path}")
    checkpoint_to_use = drive_checkpoint_path
    resume_training = True

    # Copy to local for faster training
    os.makedirs(os.path.dirname(local_checkpoint_path), exist_ok=True)
    print("Copying checkpoint from Drive to local storage...")
    shutil.copy2(drive_checkpoint_path, local_checkpoint_path)
    checkpoint_to_use = local_checkpoint_path
    print("✓ Checkpoint copied to local storage")

elif os.path.exists(local_checkpoint_path):
    print(f"\n✓ Found checkpoint locally: {local_checkpoint_path}")
    checkpoint_to_use = local_checkpoint_path
    resume_training = True

print("\n" + "="*60)
if resume_training:
    print("RESUMING TRAINING FROM CHECKPOINT")
    print(f"Checkpoint: {checkpoint_to_use}")
else:
    print("STARTING NEW TRAINING")
print("SINGLE-STAGE YOLOv5m TRAINING (70 epochs)")
print("Checkpoints saved every 10 epochs")
print(f"Drive backup: {drive_checkpoint_dir}")
print("="*60)

# Build training command with resume capability
# IMPORTANT: When resuming, we only need the checkpoint path, not --weights
if resume_training:
    # YOLOv5 resume syntax: just pass the checkpoint path
    train_cmd = f"""python -u train.py \
  --resume {checkpoint_to_use} \
  --exist-ok \
  --workers 4"""
else:
    # Fresh training with all parameters
    train_cmd = f"""python -u train.py \
  --img 640 \
  --batch {recommended_batch} \
  --epochs 70 \
  --data {config_path} \
  --weights yolov5m.pt \
  --project {model_save_dir} \
  --name single_stage \
  --save-period 10 \
  --exist-ok \
  --workers 4"""

print("\nStarting YOLOv5m training...")
print("Training command:")
print(train_cmd)
print("\nTraining Progress:\n")

# Run with progress bar display
process = subprocess.Popen(
    train_cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

import re
from IPython.display import clear_output

current_epoch = 0
last_progress_line = ""
last_backup_epoch = -1

def backup_to_drive(epoch_num):
    """Backup checkpoint to Drive"""
    if os.path.exists(local_checkpoint_path):
        try:
            print(f"\n📦 Backing up checkpoint to Drive (epoch {epoch_num})...")
            shutil.copy2(local_checkpoint_path, drive_checkpoint_path)

            # Also backup best.pt if it exists
            local_best = os.path.join(model_save_dir, 'single_stage', 'weights', 'best.pt')
            drive_best = os.path.join(drive_checkpoint_dir, 'best.pt')
            if os.path.exists(local_best):
                shutil.copy2(local_best, drive_best)

            print(f"✓ Checkpoint backed up to Drive successfully at epoch {epoch_num}!")
            return True
        except Exception as e:
            print(f"⚠ Warning: Failed to backup to Drive: {e}")
            return False
    return False

for line in process.stdout:
    # Detect epoch start
    epoch_match = re.search(r'Epoch\s+(\d+)/(\d+)', line)
    if epoch_match:
        current_epoch = int(epoch_match.group(1))
        total_epochs = int(epoch_match.group(2))
        print(f"\n{'='*60}")
        print(f"Epoch {current_epoch}/{total_epochs}")
        print('='*60)

    # Detect epoch completion (when results are saved)
    if 'Results saved to' in line or 'Epoch completed' in line:
        # BACKUP TO DRIVE EVERY 5 EPOCHS (after epoch completes)
        if current_epoch % 5 == 0 and current_epoch != last_backup_epoch and current_epoch > 0:
            last_backup_epoch = current_epoch
            backup_to_drive(current_epoch)

    # Show progress bars
    if any(x in line for x in ['%|', 'it/s', 's/it']) or re.search(r'\d+/\d+\s*\[', line):
        print(f"\r{line.strip()[:100]}", end='', flush=True)
        last_progress_line = line

    # Show important epoch summary lines
    elif any(keyword in line for keyword in ['Class', 'Images', 'Instances',
                                              'P', 'R', 'mAP50', 'mAP50-95',
                                              'all', 'Results saved']):
        if last_progress_line:
            print()
            last_progress_line = ""
        print(line.rstrip())

    # Show validation results
    elif 'val:' in line.lower() or 'validating' in line.lower():
        if last_progress_line:
            print()
            last_progress_line = ""
        print(line.rstrip())

    # Show model info and important messages
    elif any(keyword in line for keyword in ['Model summary', 'Optimizer',
                                              'Starting', 'hyperparameters',
                                              'Best', 'Saving', 'Resuming']):
        if last_progress_line:
            print()
            last_progress_line = ""
        print(line.rstrip())

print()

return_code = process.wait()

# FINAL BACKUP TO DRIVE
print("\n" + "="*60)
print("FINAL BACKUP TO DRIVE")
print("="*60)
try:
    weights_to_backup = [
        ('last.pt', 'Final checkpoint'),
        ('best.pt', 'Best model')
    ]

    for weight_file, description in weights_to_backup:
        local_path = os.path.join(model_save_dir, 'single_stage', 'weights', weight_file)
        drive_path = os.path.join(drive_checkpoint_dir, weight_file)

        if os.path.exists(local_path):
            shutil.copy2(local_path, drive_path)
            size_mb = os.path.getsize(local_path) / (1024*1024)
            print(f"✓ {description} backed up: {weight_file} ({size_mb:.1f} MB)")

    # Also backup results.csv
    results_file = os.path.join(model_save_dir, 'single_stage', 'results.csv')
    if os.path.exists(results_file):
        shutil.copy2(results_file, os.path.join(drive_checkpoint_dir, 'results.csv'))
        print("✓ Training results backed up")

    print(f"\n✓ All checkpoints backed up to: {drive_checkpoint_dir}")
except Exception as e:
    print(f"⚠ Warning: Failed to backup some files: {e}")

if return_code == 0:
    print("\n" + "="*60)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print("="*60)
else:
    print(f"\n⚠ Warning: Training ended with return code {return_code}")

# Check saved models
best_model_path = os.path.join(model_save_dir, 'single_stage', 'weights', 'best.pt')
last_model_path = os.path.join(model_save_dir, 'single_stage', 'weights', 'last.pt')
weights_dir = os.path.join(model_save_dir, 'single_stage', 'weights')

print("\nLocal Models:")
if os.path.exists(best_model_path):
    print(f"✓ Best model: {best_model_path}")
    print(f"  Size: {os.path.getsize(best_model_path) / (1024*1024):.1f} MB")

if os.path.exists(last_model_path):
    print(f"✓ Last checkpoint: {last_model_path}")
    print(f"  Size: {os.path.getsize(last_model_path) / (1024*1024):.1f} MB")

# List all epoch checkpoints
if os.path.exists(weights_dir):
    epoch_checkpoints = sorted([f for f in os.listdir(weights_dir) if f.startswith('epoch')])
    if epoch_checkpoints:
        print(f"\n✓ Periodic checkpoints ({len(epoch_checkpoints)}): {', '.join(epoch_checkpoints)}")

print("\nDrive Backup:")
if os.path.exists(drive_checkpoint_dir):
    drive_files = os.listdir(drive_checkpoint_dir)
    for f in drive_files:
        fpath = os.path.join(drive_checkpoint_dir, f)
        size_mb = os.path.getsize(fpath) / (1024*1024)
        print(f"✓ {f} ({size_mb:.1f} MB)")

print("\n" + "="*60)
print("To resume training after runtime disconnection:")
print("1. Mount your Drive")
print("2. Re-run this cell - it will automatically detect and resume!")
print("="*60)

In [ ]:
# RUN THIS CELL MANUALLY TO BACKUP CHECKPOINTS TO DRIVE
# You can run this while training is ongoing or after it completes

import shutil
import os
from datetime import datetime

# Paths
model_save_dir = '/content/yolov5_models'
drive_checkpoint_dir = '/content/drive/MyDrive/yolov5_checkpoints'

# Create Drive directory if it doesn't exist
os.makedirs(drive_checkpoint_dir, exist_ok=True)

print("="*60)
print("MANUAL CHECKPOINT BACKUP TO DRIVE")
print("="*60)

# Files to backup
files_to_backup = {
    'last.pt': 'Latest checkpoint',
    'best.pt': 'Best model',
    'results.csv': 'Training results'
}

# Also backup epoch checkpoints
weights_dir = os.path.join(model_save_dir, 'single_stage', 'weights')
if os.path.exists(weights_dir):
    epoch_files = [f for f in os.listdir(weights_dir) if f.startswith('epoch') and f.endswith('.pt')]
    for ef in epoch_files:
        files_to_backup[ef] = f'Periodic checkpoint'

backup_count = 0
for filename, description in files_to_backup.items():
    if filename == 'results.csv':
        local_path = os.path.join(model_save_dir, 'single_stage', filename)
    else:
        local_path = os.path.join(model_save_dir, 'single_stage', 'weights', filename)

    drive_path = os.path.join(drive_checkpoint_dir, filename)

    if os.path.exists(local_path):
        try:
            shutil.copy2(local_path, drive_path)
            size_mb = os.path.getsize(local_path) / (1024*1024)
            print(f"✓ {description}: {filename} ({size_mb:.1f} MB)")
            backup_count += 1
        except Exception as e:
            print(f"✗ Failed to backup {filename}: {e}")
    else:
        print(f"⚠ Not found: {filename}")

print(f"\n{'='*60}")
print(f"Backup complete! {backup_count} files saved to Drive")
print(f"Location: {drive_checkpoint_dir}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

# List all files in Drive backup location
print("\nFiles in Drive backup:")
if os.path.exists(drive_checkpoint_dir):
    drive_files = sorted(os.listdir(drive_checkpoint_dir))
    for f in drive_files:
        fpath = os.path.join(drive_checkpoint_dir, f)
        size_mb = os.path.getsize(fpath) / (1024*1024)
        mtime = datetime.fromtimestamp(os.path.getmtime(fpath))
        print(f"  • {f} ({size_mb:.1f} MB) - {mtime.strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("  No files found")

In [ ]:
# Cell 7: YOLOv5 Evaluation
import torch
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.torch_utils import select_device

def load_yolov5_model(weights_path):
    """Load YOLOv5 model"""
    device = select_device('0' if torch.cuda.is_available() else 'cpu')
    model = DetectMultiBackend(weights_path, device=device, dnn=False, data=config_path, fp16=False)
    model.eval()
    return model, device

def detect_yolov5(model, device, img_path, conf_thresh=0.5, img_size=640):
    """Run YOLOv5 detection"""
    from utils.dataloaders import LoadImages
    from utils.augmentations import letterbox

    # Load image
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # Prepare input - letterbox resize like YOLOv5 does
    img_input = letterbox(img_rgb, img_size, stride=32, auto=True)[0]
    img_input = img_input.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_input = np.ascontiguousarray(img_input)
    img_input = torch.from_numpy(img_input).to(device)
    img_input = img_input.float()
    img_input /= 255.0

    if img_input.ndimension() == 3:
        img_input = img_input.unsqueeze(0)

    # Inference
    with torch.no_grad():
        pred = model(img_input, augment=False, visualize=False)

    # NMS
    pred = non_max_suppression(pred, conf_thresh, 0.45, classes=None, agnostic=False, max_det=1000)

    # Process detections
    boxes = []
    if pred[0] is not None and len(pred[0]):
        det = pred[0]
        det[:, :4] = scale_boxes(img_input.shape[2:], det[:, :4], img.shape).round()

        for *xyxy, conf, cls in det:
            x1, y1, x2, y2 = [float(x) for x in xyxy]
            boxes.append([x1, y1, x2, y2])

    return boxes

def load_ground_truth_boxes(label_path, img_width, img_height):
    """Load ground truth boxes from YOLO format label file"""
    boxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line:
                parts = line.split()
                if len(parts) == 5:
                    try:
                        _, x_center, y_center, width, height = map(float, parts)

                        x_center_abs = x_center * img_width
                        y_center_abs = y_center * img_height
                        width_abs = width * img_width
                        height_abs = height * img_height

                        x1 = x_center_abs - width_abs / 2
                        y1 = y_center_abs - height_abs / 2
                        x2 = x_center_abs + width_abs / 2
                        y2 = y_center_abs + height_abs / 2

                        boxes.append([x1, y1, x2, y2])
                    except ValueError:
                        continue
    return boxes

def calculate_box_iou(box1, box2):
    """Calculate IoU between two bounding boxes"""
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)

    if x2_i <= x1_i or y2_i <= y1_i:
        return 0.0

    intersection_area = (x2_i - x1_i) * (y2_i - y1_i)
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    union_area = box1_area + box2_area - intersection_area

    if union_area == 0:
        return 0.0

    return intersection_area / union_area

def evaluate_yolov5(model, device, split_name, conf_thresh=0.5):
    """Comprehensive evaluation with same metrics"""
    img_dir = f'{yolo_dataset_path}/{split_name}/images'
    label_dir = f'{yolo_dataset_path}/{split_name}/labels'

    img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]

    tp, fp, fn, tn = 0, 0, 0, 0
    iou_scores = []

    print(f"Evaluating {len(img_files)} images...")

    for i, img_file in enumerate(img_files):
        if i % 500 == 0 and i > 0:
            print(f"Progress: {i}/{len(img_files)} ({i/len(img_files)*100:.1f}%)")

        img_path = f'{img_dir}/{img_file}'
        label_path = f'{label_dir}/{os.path.splitext(img_file)[0]}.txt'

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w = img.shape[:2]

        gt_boxes = load_ground_truth_boxes(label_path, w, h)
        has_gt = len(gt_boxes) > 0

        pred_boxes = detect_yolov5(model, device, img_path, conf_thresh)
        has_pred = len(pred_boxes) > 0

        if has_gt and has_pred:
            max_iou = 0
            for gt_box in gt_boxes:
                for pred_box in pred_boxes:
                    iou = calculate_box_iou(gt_box, pred_box)
                    max_iou = max(max_iou, iou)

            iou_scores.append(max_iou)
            if max_iou > 0.5:
                tp += 1
            else:
                fn += 1
        elif has_gt and not has_pred:
            fn += 1
            iou_scores.append(0)
        elif not has_gt and has_pred:
            fp += 1
            iou_scores.append(0)
        else:
            tn += 1
            iou_scores.append(1)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / len(img_files)
    avg_iou = sum(iou_scores) / len(iou_scores)
    binary_detection = sum(1 for iou in iou_scores if iou > 0.7) / len(iou_scores)

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'avg_iou': avg_iou,
        'binary_detection': binary_detection,
        'iou_scores': iou_scores
    }

# Load the trained model
best_model_path = os.path.join(model_save_dir, 'single_stage', 'weights', 'best.pt')
print(f"Loading model from: {best_model_path}")
trained_model, device = load_yolov5_model(best_model_path)
print("Model loaded successfully!")

# Evaluate the model
print("\nEvaluating YOLOv5m model...")
val_results = evaluate_yolov5(trained_model, device, 'validation')
test_results = evaluate_yolov5(trained_model, device, 'test')

print("\n" + "="*50)
print("YOLOv5m SINGLE-STAGE RESULTS")
print("="*50)

print("\nVALIDATION RESULTS:")
print(f"IoU: {val_results['avg_iou']:.4f}")
print(f"Precision: {val_results['precision']:.4f}")
print(f"Recall: {val_results['recall']:.4f}")
print(f"F1-Score: {val_results['f1_score']:.4f}")
print(f"Binary Detection (IoU>0.7): {val_results['binary_detection']:.4f}")

print("\nTEST RESULTS:")
print(f"IoU: {test_results['avg_iou']:.4f}")
print(f"Precision: {test_results['precision']:.4f}")
print(f"Recall: {test_results['recall']:.4f}")
print(f"F1-Score: {test_results['f1_score']:.4f}")
print(f"Binary Detection (IoU>0.7): {test_results['binary_detection']:.4f}")
print(f"Boundary Box Accuracy: {test_results['binary_detection']*100:.1f}%")

# Cell 8: Training Visualization for YOLOv5
results_csv = f'{model_save_dir}/single_stage/results.csv'

if os.path.exists(results_csv):
    data = pd.read_csv(results_csv)
    # Strip whitespace from column names
    data.columns = data.columns.str.strip()

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    epochs = range(1, len(data) + 1)

    # Box Loss - YOLOv5 column names
    if 'train/box_loss' in data.columns:
        axes[0,0].plot(epochs, data['train/box_loss'], 'b-', label='Train Box Loss')
    if 'val/box_loss' in data.columns:
        axes[0,0].plot(epochs, data['val/box_loss'], 'r-', label='Val Box Loss')
    axes[0,0].set_title('Box Loss')
    axes[0,0].set_xlabel('Epoch')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    axes[0,0].grid(True)

    # mAP@0.5
    if 'metrics/mAP_0.5' in data.columns:
        axes[0,1].plot(epochs, data['metrics/mAP_0.5'], 'g-', label='mAP@0.5')
        axes[0,1].set_title('mAP@0.5')
        axes[0,1].set_xlabel('Epoch')
        axes[0,1].set_ylabel('mAP')
        axes[0,1].legend()
        axes[0,1].grid(True)

    # Precision & Recall
    if 'metrics/precision' in data.columns and 'metrics/recall' in data.columns:
        axes[1,0].plot(epochs, data['metrics/precision'], 'purple', label='Precision')
        axes[1,0].plot(epochs, data['metrics/recall'], 'brown', label='Recall')
        axes[1,0].set_title('Precision & Recall')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('Score')
        axes[1,0].legend()
        axes[1,0].grid(True)

    # Final metrics comparison
    metrics = ['IoU', 'Precision', 'Recall', 'F1', 'Binary Det']
    test_scores = [test_results['avg_iou'], test_results['precision'],
                   test_results['recall'], test_results['f1_score'],
                   test_results['binary_detection']]

    axes[1,1].bar(metrics, test_scores, alpha=0.7, color=['blue', 'green', 'red', 'orange', 'purple'])
    axes[1,1].set_title('Final Test Performance')
    axes[1,1].set_xticklabels(metrics, rotation=45)
    axes[1,1].set_ylim([0, 1])
    axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{model_save_dir}/single_stage/training_metrics.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\nTraining Summary:")
    print(f"Total epochs: {len(data)}")
    if 'metrics/mAP_0.5' in data.columns:
        print(f"Best mAP@0.5: {data['metrics/mAP_0.5'].max():.4f}")
    print(f"Final Test IoU: {test_results['avg_iou']:.4f}")
else:
    print(f"Results file not found at: {results_csv}")

print("\n" + "="*60)
print("YOLOv5m single-stage training completed!")
print("="*60)

In [ ]:
import shutil
from google.colab import files, drive



# Specify the folder path you want to zip
folder_path = '/content/yolov5_models'
zip_name = 'yolomodel'

# Create a zip file of the folder
shutil.make_archive(zip_name, 'zip', folder_path)

# Save to Google Drive
drive_path = '/content/drive/MyDrive/yolomodel.zip'
shutil.move(f'{zip_name}.zip', drive_path)
print(f"Saved to Google Drive: {drive_path}")

# Also download to your local machine
#shutil.make_archive(zip_name, 'zip', folder_path)
#files.download(f'{zip_name}.zip')
print(f"Downloaded {zip_name}.zip to your computer successfully!")